In [82]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

Importing the data

In [83]:
dataset = pd.read_csv('https://raw.githubusercontent.com/revanthmadasu/LifestyleWellbeing/master/Wellbeing_and_lifestyle_data_Kaggle.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Converting categorial data into numerical

Using method for 'GENDER' category

In [90]:
"""# Converting type of columns to category
dataset['GENDER']=dataset['GENDER'].astype('category')

#Assigning numerical values and storing it in another columns
dataset['Gen_new']=dataset['GENDER'].cat.codes

#Create an instance of One-hot-encoder
onehotencoder=OneHotEncoder()

onehotencoder_dataset=pd.DataFrame(onehotencoder.fit_transform(dataset[['Gen_new']]).toarray())

#Merge with main
New_dataset=dataset.join(onehotencoder_dataset)
New_dataset.columns
print(New_dataset)"""

"# Converting type of columns to category\ndataset['GENDER']=dataset['GENDER'].astype('category')\n\n#Assigning numerical values and storing it in another columns\ndataset['Gen_new']=dataset['GENDER'].cat.codes\n\n#Create an instance of One-hot-encoder\nonehotencoder=OneHotEncoder()\n\nonehotencoder_dataset=pd.DataFrame(onehotencoder.fit_transform(dataset[['Gen_new']]).toarray())\n\n#Merge with main\nNew_dataset=dataset.join(onehotencoder_dataset)\nNew_dataset.columns\nprint(New_dataset)"

In [97]:
mdataset = dataset[dataset.columns[0:-1]]
one_hot_encoded_data = pd.get_dummies(mdataset, columns = ['GENDER'])
print(one_hot_encoded_data)

          Timestamp  FRUITS_VEGGIES DAILY_STRESS  PLACES_VISITED  CORE_CIRCLE  \
0            7/7/15               3            2               2            5   
1            7/7/15               2            3               4            3   
2            7/7/15               2            3               3            4   
3            7/7/15               3            3              10            3   
4            7/7/15               5            1               3            3   
...             ...             ...          ...             ...          ...   
15967  3/14/21 5:42               3            3               0            4   
15968  3/14/21 6:30               3            3               6            8   
15969  3/14/21 8:35               4            3               0           10   
15970  3/14/21 8:43               1            1              10            8   
15971  3/14/21 9:03               5            4               0            2   

       SUPPORTING_OTHERS  S

In [87]:
print(X[0,:])

['7/7/15' 3 '2' 2 5 0 5 2 0 1 6 4 5 0 7 5 5 1 4 0 5 '36 to 50' 'Female']


In [91]:
print(X)

[['7/7/15' 3 '2' ... 5 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 6 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 3 '36 to 50' 'Female']
 ...
 ['3/14/21 8:35' 4 '3' ... 10 '21 to 35' 'Male']
 ['3/14/21 8:43' 1 '1' ... 8 '21 to 35' 'Female']
 ['3/14/21 9:03' 5 '4' ... 4 '21 to 35' 'Female']]


In [89]:
print(y)

[609.5 655.6 631.6 ... 716.6 682.  651.4]


Handling missing data

In [ ]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer.fit(X[:, 1:3])
# X[:, 1:3] = imputer.transform(X[:, 1:3])